In [54]:
# ================================================
# 02_feature_engineering.ipynb
# Feature generation for NBA game prediction model
# ================================================

In [55]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set paths
DATA_RAW = Path("../data/raw")
DATA_FINAL = Path("../data/final")

# Display all columns for debugging
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)

In [56]:
# ===============================================================
# 1. Load cleaned datasets
# ===============================================================

# --- NEW, CORRECTED CODE ---

# Tell pandas to automatically parse the date column as it loads the data
games = pd.read_csv(DATA_RAW / "Games.csv", parse_dates=['gameDate'])
team_stats = pd.read_csv(DATA_RAW / "TeamStatistics.csv", parse_dates=['gameDate'])
player_stats = pd.read_csv(DATA_RAW / "PlayerStatistics.csv", parse_dates=['gameDate'])

print(f"Loaded datasets | Games: {games.shape}, TeamStats: {team_stats.shape}, PlayerStats: {player_stats.shape}")

# VALIDATION: Check for missing dates in the original team_stats file right after loading
print(f"\nMissing dates in original team_stats file: {team_stats['gameDate'].isna().sum()}")

/var/folders/l3/gv_dtzgn653g0kctx7f1g_0w0000gn/T/ipykernel_1889/3662626992.py:8: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv(DATA_RAW / "Games.csv", parse_dates=['gameDate'])
/var/folders/l3/gv_dtzgn653g0kctx7f1g_0w0000gn/T/ipykernel_1889/3662626992.py:10: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv(DATA_RAW / "PlayerStatistics.csv", parse_dates=['gameDate'])


Loaded datasets | Games: (72053, 17), TeamStats: (144106, 48), PlayerStats: (1632752, 35)

Missing dates in original team_stats file: 0


In [57]:
# ===============================================================
# 2. Aggregate player statistics to team level
# ===============================================================

# Detect correct gameId column automatically
possible_gameid_cols = [c for c in player_stats.columns if "gameid" in c.lower()]
gameid_col = possible_gameid_cols[0] if possible_gameid_cols else None
if not gameid_col:
    raise KeyError("No 'gameId' column found in player_stats")

# Aggregate player-level stats per team per game
player_team_agg = (
    player_stats.groupby([gameid_col, "playerteamName"])
    .agg({
        "points": "sum",
        "assists": "sum",
        "reboundsTotal": "sum",
        "steals": "sum",
        "blocks": "sum",
        "numMinutes": "mean",
        "fieldGoalsPercentage": "mean",
        "threePointersPercentage": "mean",
        "freeThrowsPercentage": "mean",
    })
    .reset_index()
    .rename(columns={gameid_col: "gameId", "playerteamName": "teamName"})
)

print(f"Aggregated player stats: {player_team_agg.shape}")
display(player_team_agg.head(3))

Aggregated player stats: (144102, 11)


,gameId,teamName,points,assists,reboundsTotal,steals,blocks,numMinutes,fieldGoalsPercentage,threePointersPercentage,freeThrowsPercentage
0,10300001,Jazz,90.0,23.0,41.0,8.0,4.0,5.0,0.513474,0.052632,0.430579
1,10300001,Mavericks,85.0,20.0,38.0,9.0,4.0,5.0,0.379182,0.090909,0.290000
2,10300002,Bucks,94.0,20.0,43.0,9.0,4.0,5.0,0.347462,0.038462,0.491538


In [58]:
# ===============================================================
# 3. Standardize key columns in team_stats before merge
# ===============================================================

# Strip accidental spaces
team_stats.columns = team_stats.columns.str.strip()

# Auto-detect and rename gameId column
possible_gameid_cols = [c for c in team_stats.columns if "gameid" in c.lower()]
if possible_gameid_cols:
    old_col = possible_gameid_cols[0]
    team_stats.rename(columns={old_col: "gameId"}, inplace=True)
else:
    raise KeyError("No 'gameId' column found in team_stats!")

# Auto-detect and rename teamName column
possible_teamname_cols = [c for c in team_stats.columns if "teamname" in c.lower()]
if possible_teamname_cols:
    old_col = possible_teamname_cols[0]
    team_stats.rename(columns={old_col: "teamName"}, inplace=True)
else:
    raise KeyError("No 'teamName' column found in team_stats!")

print("Standardized key columns in team_stats:")
display(team_stats[["gameId", "teamName"]].head())


Standardized key columns in team_stats:


,gameId,teamName
0,22500164,Clippers
1,22500164,Heat
2,22500163,Lakers
3,22500163,Trail Blazers
4,22500162,Nuggets


In [59]:
# ===============================================================
# 4. Merge aggregated player stats with team stats
# ===============================================================

# Merge team-level official stats with aggregated player-level stats
merged_team = pd.merge(
    team_stats,
    player_team_agg,
    how="left",
    on=["gameId", "teamName"],
    suffixes=("", "_players")
)

# Remove duplicates
merged_team.drop_duplicates(subset=["gameId", "teamName"], inplace=True)

# Sanity checks
missing_teams = merged_team["gameId"].nunique() - team_stats["gameId"].nunique()
if missing_teams == 0:
    print(f"Merged dataset OK: {merged_team.shape}")
else:
    print(f"{abs(missing_teams)} games may be missing after merge.")

# Validation checks
print("\nNull values per key column:")
print(merged_team[["gameId", "teamName"]].isna().sum())

print("\nDuplicated gameId–teamName pairs:", 
      merged_team.duplicated(subset=["gameId", "teamName"]).sum())

display(merged_team.head(3))

Merged dataset OK: (144106, 57)

Null values per key column:
gameId      0
teamName    0
dtype: int64

Duplicated gameId–teamName pairs: 0


,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,teamScore,opponentScore,assists,blocks,steals,fieldGoalsAttempted,fieldGoalsMade,fieldGoalsPercentage,threePointersAttempted,threePointersMade,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,numMinutes,q1Points,q2Points,q3Points,q4Points,benchPoints,biggestLead,biggestScoringRun,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId,points,assists_players,reboundsTotal_players,steals_players,blocks_players,numMinutes_players,fieldGoalsPercentage_players,threePointersPercentage_players,freeThrowsPercentage_players
0,22500164,2025-11-03T22:30:00Z,LA,Clippers,1610612746,Miami,Heat,1610612748,1,0,119,120,25.0,4.0,13.0,88.0,44.0,0.500,41.0,17.0,0.415,19.0,14.0,0.737,33.0,11.0,44.0,24.0,20.0,-1.0,240.0,30.0,41.0,24.0,24.0,30.0,11.0,13.0,18.0,14.0,25.0,48.0,18.0,9.0,0.0,3.0,3.0,NaN,119.0,25.0,44.0,13.0,4.0,23.840000,0.322786,0.261929,0.214286
1,22500164,2025-11-03T22:30:00Z,Miami,Heat,1610612748,LA,Clippers,1610612746,0,1,120,119,33.0,3.0,12.0,83.0,45.0,0.542,25.0,12.0,0.480,24.0,18.0,0.750,31.0,6.0,37.0,15.0,16.0,1.0,240.0,32.0,34.0,37.0,17.0,42.0,13.0,10.0,18.0,11.0,37.0,58.0,7.0,9.0,0.0,4.0,3.0,NaN,120.0,33.0,37.0,12.0,3.0,23.760000,0.444417,0.313917,0.404167
2,22500163,2025-11-03T22:00:00Z,Los Angeles,Lakers,1610612747,Portland,Trail Blazers,1610612757,0,1,123,115,29.0,5.0,11.0,85.0,50.0,0.588,23.0,9.0,0.391,16.0,14.0,0.875,26.0,8.0,34.0,22.0,17.0,8.0,240.0,24.0,28.0,38.0,33.0,38.0,14.0,10.0,2.0,8.0,25.0,64.0,20.0,2.0,1.0,6.0,2.0,NaN,123.0,29.0,34.0,11.0,5.0,26.444444,0.444333,0.169417,0.416667


In [60]:
# ===============================================================
# 5. Proceed to compute feature differentials
# ===============================================================

print("\nReady for differential feature computation.")
print(f"Total unique gameIds: {merged_team['gameId'].nunique()}")


Ready for differential feature computation.
Total unique gameIds: 72053


In [61]:
# ===============================================================
# 6. Compute feature differentials (home – away)
# ===============================================================

def diff_features(df, prefix_home, prefix_away, columns):
    """
    Compute (home - away) feature differences for the selected columns.
    Returns a DataFrame with 'diff_' prefixed columns.
    """
    diff_df = pd.DataFrame()
    for col in columns:
        home_col = f"{prefix_home}_{col}"
        away_col = f"{prefix_away}_{col}"
        if home_col in df.columns and away_col in df.columns:
            diff_df[f"diff_{col}"] = df[home_col] - df[away_col]
        else:
            print(f"Missing columns for: {col}")
    return diff_df


# --- Split dataset into home and away subsets ---
home = merged_team[merged_team["home"] == 1].add_prefix("home_")
away = merged_team[merged_team["home"] == 0].add_prefix("away_")

# --- Merge home and away by gameId ---
combined = pd.merge(
    home,
    away,
    left_on="home_gameId",
    right_on="away_gameId",
    how="inner",
    suffixes=("_home", "_away")
)

# --- Compute statistical differentials (team + player aggregates) ---
stats_cols = [
    "points", "assists", "reboundsTotal", "steals", "blocks",
    "fieldGoalsPercentage", "threePointersPercentage", "freeThrowsPercentage",
    "assists_players", "reboundsTotal_players", "steals_players", "blocks_players"
]

diffs = diff_features(combined, "home", "away", stats_cols)

# Combine base game info + differentials
final_df = pd.concat([combined, diffs], axis=1)

# --- Integrity check ---
expected_games = merged_team["gameId"].nunique()
actual_games = final_df["home_gameId"].nunique()
if expected_games == actual_games:
    print(f"Feature differential dataset created successfully.")
else:
    print(f"Mismatch: expected {expected_games}, found {actual_games} after merge.")

print(f"Shape: {final_df.shape}")
display(final_df.head(3))

# --- Cleanup redundant columns ---
final_df.drop(columns=["away_gameId", "away_gameDate"], inplace=True, errors="ignore")
print("Cleaned redundant columns: ['away_gameId', 'away_gameDate']")

Feature differential dataset created successfully.
Shape: (72053, 126)


,home_gameId,home_gameDate,home_teamCity,home_teamName,home_teamId,home_opponentTeamCity,home_opponentTeamName,home_opponentTeamId,home_home,home_win,home_teamScore,home_opponentScore,home_assists,home_blocks,home_steals,home_fieldGoalsAttempted,home_fieldGoalsMade,home_fieldGoalsPercentage,home_threePointersAttempted,home_threePointersMade,home_threePointersPercentage,home_freeThrowsAttempted,home_freeThrowsMade,home_freeThrowsPercentage,home_reboundsDefensive,home_reboundsOffensive,home_reboundsTotal,home_foulsPersonal,home_turnovers,home_plusMinusPoints,home_numMinutes,home_q1Points,home_q2Points,home_q3Points,home_q4Points,home_benchPoints,home_biggestLead,home_biggestScoringRun,home_leadChanges,home_pointsFastBreak,home_pointsFromTurnovers,home_pointsInThePaint,home_pointsSecondChance,home_timesTied,home_timeoutsRemaining,home_seasonWins,home_seasonLosses,home_coachId,home_points,home_assists_players,home_reboundsTotal_players,home_steals_players,home_blocks_players,home_numMinutes_players,home_fieldGoalsPercentage_players,home_threePointersPercentage_players,home_freeThrowsPercentage_players,away_gameId,away_gameDate,away_teamCity,away_teamName,away_teamId,away_opponentTeamCity,away_opponentTeamName,away_opponentTeamId,away_home,away_win,away_teamScore,away_opponentScore,away_assists,away_blocks,away_steals,away_fieldGoalsAttempted,away_fieldGoalsMade,away_fieldGoalsPercentage,away_threePointersAttempted,away_threePointersMade,away_threePointersPercentage,away_freeThrowsAttempted,away_freeThrowsMade,away_freeThrowsPercentage,away_reboundsDefensive,away_reboundsOffensive,away_reboundsTotal,away_foulsPersonal,away_turnovers,away_plusMinusPoints,away_numMinutes,away_q1Points,away_q2Points,away_q3Points,away_q4Points,away_benchPoints,away_biggestLead,away_biggestScoringRun,away_leadChanges,away_pointsFastBreak,away_pointsFromTurnovers,away_pointsInThePaint,away_pointsSecondChance,away_timesTied,away_timeoutsRemaining,away_seasonWins,away_seasonLosses,away_coachId,away_points,away_assists_players,away_reboundsTotal_players,away_steals_players,away_blocks_players,away_numMinutes_players,away_fieldGoalsPercentage_players,away_threePointersPercentage_players,away_freeThrowsPercentage_players,diff_points,diff_assists,diff_reboundsTotal,diff_steals,diff_blocks,diff_fieldGoalsPercentage,diff_threePointersPercentage,diff_freeThrowsPercentage,diff_assists_players,diff_reboundsTotal_players,diff_steals_players,diff_blocks_players
0,22500164,2025-11-03T22:30:00Z,LA,Clippers,1610612746,Miami,Heat,1610612748,1,0,119,120,25.0,4.0,13.0,88.0,44.0,0.500,41.0,17.0,0.415,19.0,14.0,0.737,33.0,11.0,44.0,24.0,20.0,-1.0,240.0,30.0,41.0,24.0,24.0,30.0,11.0,13.0,18.0,14.0,25.0,48.0,18.0,9.0,0.0,3.0,3.0,NaN,119.0,25.0,44.0,13.0,4.0,23.84,0.322786,0.261929,0.214286,22500164,2025-11-03T22:30:00Z,Miami,Heat,1610612748,LA,Clippers,1610612746,0,1,120,119,33.0,3.0,12.0,83.0,45.0,0.542,25.0,12.0,0.480,24.0,18.0,0.750,31.0,6.0,37.0,15.0,16.0,1.0,240.0,32.0,34.0,37.0,17.0,42.0,13.0,10.0,18.0,11.0,37.0,58.0,7.0,9.0,0.0,4.0,3.0,NaN,120.0,33.0,37.0,12.0,3.0,23.760000,0.444417,0.313917,0.404167,-1.0,-8.0,7.0,1.0,1.0,-0.042,-0.065,-0.013,-8.0,7.0,1.0,1.0
1,22500163,2025-11-03T22:00:00Z,Portland,Trail Blazers,1610612757,Los Angeles,Lakers,1610612747,1,0,115,123,23.0,3.0,13.0,85.0,42.0,0.494,40.0,9.0,0.225,30.0,22.0,0.733,23.0,15.0,38.0,18.0,18.0,-8.0,240.0,33.0,20.0,33.0,29.0,35.0,13.0,9.0,2.0,20.0,28.0,64.0,26.0,2.0,1.0,4.0,3.0,NaN,115.0,23.0,38.0,13.0,3.0,23.76,0.350200,0.094400,0.215333,22500163,2025-11-03T22:00:00Z,Los Angeles,Lakers,1610612747,Portland,Trail Blazers,1610612757,0,1,123,115,29.0,5.0,11.0,85.0,50.0,0.588,23.0,9.0,0.391,16.0,14.0,0.875,26.0,8.0,34.0,22.0,17.0,8.0,240.0,24.0,28.0,38.0,33.0,38.0,14.0,10.0,2.0,8.0,25.0,64.0,20.0,2.0,1.0,6.0,2.0,NaN,123.0,29.0,34.0,11.0,5.0,26.444444,0.444333,0.169417,0.416667,-8.0,-6.0,4.0,2.0,-2.0,-0.094,-0.166,-0.142,-6.0,4.0,2.0,-2.0
2,22500162,2025-11-03T21:00:00Z,Denver,Nuggets,1610612743,Sacramento,Kings,161061

Cleaned redundant columns: ['away_gameId', 'away_gameDate']


In [62]:
# ===============================================================
# 7. Target variables
# ===============================================================

# Define binary win target and continuous score differential
final_df["home_win"] = (final_df["home_points"] > final_df["away_points"]).astype(int)
final_df["score_diff"] = final_df["home_points"] - final_df["away_points"]

print("Target variables created successfully.")
display(final_df[["home_points", "away_points", "home_win", "score_diff"]].head())

Target variables created successfully.


,home_points,away_points,home_win,score_diff
0,119.0,120.0,0,-1.0
1,115.0,123.0,0,-8.0
2,130.0,124.0,1,6.0
3,106.0,114.0,0,-8.0
4,110.0,102.0,1,8.0


In [63]:
# ===============================================================
# 8. Add contextual and categorical features
# ===============================================================
# Adds contextual metadata about game type and environment.
# Handles cases where 'gameType' column may have variant names (e.g., 'home_gameType').

# --- 1. Detect home game type column dynamically ---
game_type_cols = [c for c in final_df.columns if "gameType" in c and "home" in c]

if game_type_cols:
    col = game_type_cols[0]
    print(f"📂 Detected game type column: {col}")
    
    # Create categorical flags
    final_df["is_playoffs"] = (final_df[col].str.contains("Playoff", case=False, na=False)).astype(int)
    final_df["is_regular"] = (final_df[col].str.contains("Regular", case=False, na=False)).astype(int)
else:
    print("No 'gameType' column found — assigning default values.")
    final_df["is_playoffs"] = 0
    final_df["is_regular"] = 0


# --- 2. Add contextual variables such as attendance (if exists) ---
context_cols = [
    "home_win",
    "score_diff",
    "is_playoffs",
    "is_regular",
]

attendance_cols = [c for c in final_df.columns if "attendance" in c and "home" in c]
if attendance_cols:
    final_df["home_attendance"] = final_df[attendance_cols[0]]
    context_cols.append("home_attendance")
    print("Attendance column added.")
else:
    print("No home attendance column found — skipping.")


# --- 3. Confirm contextual feature creation ---
print(f"\nContextual features added: {len(context_cols)}")
print(f"Included columns: {context_cols}")
display(final_df[context_cols].head())

No 'gameType' column found — assigning default values.
No home attendance column found — skipping.

Contextual features added: 4
Included columns: ['home_win', 'score_diff', 'is_playoffs', 'is_regular']


,home_win,score_diff,is_playoffs,is_regular
0,0,-1.0,0,0
1,0,-8.0,0,0
2,1,6.0,0,0
3,0,-8.0,0,0
4,1,8.0,0,0


In [64]:
# ===============================================================
# 9. Rolling performance indicators (momentum features)
# ===============================================================
# Adds rolling 5-game averages for each team to capture performance trends.

def add_rolling_features(df, team_col, feature_cols, window=5):
    """
    Compute rolling averages for selected statistical features by team.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing per-game stats.
    team_col : str
        Column identifying the team (e.g., 'home_teamName').
    feature_cols : list
        List of columns to compute rolling averages for.
    window : int, default=5
        Number of previous games to include in the moving average.
    
    Returns
    -------
    df : pd.DataFrame
        Dataframe with new rolling average columns added.
    """
    # Identify a game date column dynamically
    date_cols = [c for c in df.columns if "gameDate" in c]
    if not date_cols:
        raise KeyError("No date column found to compute rolling features.")
    date_col = date_cols[0]

    # Ensure chronological order
    df = df.sort_values(by=[team_col, date_col])

    # Compute rolling means for each feature
    for col in feature_cols:
        if col not in df.columns:
            print(f"⚠️ Skipping missing column: {col}")
            continue

        new_col = f"{col}_rolling{window}"
        df[new_col] = (
            df.groupby(team_col)[col]
              .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
        )

    return df


# --- Apply rolling averages for both home and away teams ---
window = 5  # rolling window size (define it here once)

rolling_features = [
    "home_points", "home_assists", "home_reboundsTotal",
    "home_fieldGoalsPercentage", "home_threePointersPercentage", "home_freeThrowsPercentage",
    "away_points", "away_assists", "away_reboundsTotal",
    "away_fieldGoalsPercentage", "away_threePointersPercentage", "away_freeThrowsPercentage"
]

final_df = add_rolling_features(final_df, team_col="home_teamName", feature_cols=rolling_features, window=5)
final_df = add_rolling_features(final_df, team_col="away_teamName", feature_cols=rolling_features, window=5)

print(f"Added rolling performance features ({len(rolling_features)} base columns × 2 teams).")
print(f"Rolling window size: {window} games.")
print(f"New rolling columns created: {[f'{col}_rolling5' for col in rolling_features]}")

Added rolling performance features (12 base columns × 2 teams).
Rolling window size: 5 games.
New rolling columns created: ['home_points_rolling5', 'home_assists_rolling5', 'home_reboundsTotal_rolling5', 'home_fieldGoalsPercentage_rolling5', 'home_threePointersPercentage_rolling5', 'home_freeThrowsPercentage_rolling5', 'away_points_rolling5', 'away_assists_rolling5', 'away_reboundsTotal_rolling5', 'away_fieldGoalsPercentage_rolling5', 'away_threePointersPercentage_rolling5', 'away_freeThrowsPercentage_rolling5']


In [65]:
# ===============================================================
# 10. Merge full game schedule to recover missing dates
# ===============================================================

import pandas as pd

# 1. Load official Games.csv
games_df = pd.read_csv("../data/raw/Games.csv", usecols=["gameId", "gameDate"])
games_df["gameId"] = pd.to_numeric(games_df["gameId"], errors="coerce").astype("Int64")
games_df["gameDate"] = pd.to_datetime(games_df["gameDate"], errors="coerce", utc=True)

print("Loaded Games.csv:", len(games_df), "rows")
print("Non-null gameDate:", games_df["gameDate"].notna().sum())

# 2. Ensure final_df has a valid gameId column
if "gameId" not in final_df.columns:
    if "home_gameId" in final_df.columns:
        final_df["gameId"] = final_df["home_gameId"]
        print("Recovered 'gameId' from 'home_gameId'.")
    elif "away_gameId" in final_df.columns:
        final_df["gameId"] = final_df["away_gameId"]
        print("Recovered 'gameId' from 'away_gameId'.")
    else:
        raise KeyError("Missing 'gameId', 'home_gameId', and 'away_gameId' in final_df.")

final_df["gameId"] = pd.to_numeric(final_df["gameId"], errors="coerce").astype("Int64")

# Remove any existing 'gameDate' column to avoid conflicts
if "gameDate" in final_df.columns:
    final_df.drop(columns=["gameDate"], inplace=True)
    print("Removed existing 'gameDate' column before merging.")

# 3. Merge using gameId
merged = final_df.merge(games_df, how="left", on="gameId", indicator=True)
print("\nMerge indicator summary:")
print(merged["_merge"].value_counts())

# 4. Fill missing home_gameDate values
if "home_gameDate" not in merged.columns:
    merged["home_gameDate"] = pd.NaT

mask_fill = merged["home_gameDate"].isna() & merged["gameDate"].notna()
filled_count = mask_fill.sum()

merged.loc[mask_fill, "home_gameDate"] = merged.loc[mask_fill, "gameDate"]
print(f"\nFilled {filled_count} missing home_gameDate values from Games.csv")

# Cleanup
merged.drop(columns=["gameDate", "_merge"], inplace=True, errors="ignore")

# 5. Validation
missing_after = merged["home_gameDate"].isna().sum()
print(f"Missing home_gameDate after merge: {missing_after}")

# 6. Preview sample
print("\nSample of merged rows:")
display(merged[["home_teamName", "away_teamName", "home_gameDate"]].sample(5, random_state=42))

# Update reference
final_df = merged

Loaded Games.csv: 72053 rows
Non-null gameDate: 174
Recovered 'gameId' from 'home_gameId'.

Merge indicator summary:
_merge
both          72053
left_only         0
right_only        0
Name: count, dtype: int64

Filled 0 missing home_gameDate values from Games.csv
Missing home_gameDate after merge: 0

Sample of merged rows:


,home_teamName,away_teamName,home_gameDate
50220,76ers,Pistons,1999-02-09 19:00:00
46188,Cavaliers,Pacers,1995-02-04 20:00:00
25356,Cavaliers,Hornets,2015-01-23 19:30:00
46574,Kings,Pacers,2003-12-07 21:00:00
8044,Knicks,Bulls,1977-01-25 19:00:00


In [66]:
# ===============================================================
# 11. Cleanup, export, and validation
# ===============================================================

import numpy as np

# 1. Identify rolling columns
rolling_cols = [c for c in final_df.columns if "_rolling" in c]

# 2. Define export structure
base_cols = [
    "home_win", "score_diff",
    "diff_points", "diff_assists", "diff_reboundsTotal",
    "diff_steals", "diff_blocks",
    "diff_fieldGoalsPercentage", "diff_threePointersPercentage", "diff_freeThrowsPercentage",
    "is_playoffs", "is_regular",
    "home_teamName", "away_teamName", "home_gameDate"
]
export_cols = base_cols + rolling_cols

# 3. Filter to existing columns
available_cols = [c for c in export_cols if c in final_df.columns]
final_dataset = final_df[available_cols].copy()

# 4. Handle missing values
numeric_cols = final_dataset.select_dtypes(include=[np.number]).columns
final_dataset[numeric_cols] = final_dataset[numeric_cols].fillna(final_dataset[numeric_cols].median())

# 5. Sort chronologically
if "home_gameDate" in final_dataset.columns:
    final_dataset["home_gameDate"] = pd.to_datetime(final_dataset["home_gameDate"], errors="coerce")
    final_dataset = final_dataset.sort_values(by="home_gameDate").reset_index(drop=True)

# 6. Validation summary
print("Final dataset ready for export.")
print(f"Shape: {final_dataset.shape}")
print(f"Columns included: {len(final_dataset.columns)}")
print(f"Rolling features detected: {len(rolling_cols)}")

# 7. Missing-value summary
na_summary = final_dataset.isna().sum()
if na_summary.sum() > 0:
    print("\nColumns with missing values:")
    print(na_summary[na_summary > 0])
else:
    print("\nNo missing values detected.")
# --- FINAL CLEANUP: Drop rows with no date ---

rows_before = len(final_dataset)
final_dataset.dropna(subset=['home_gameDate'], inplace=True)
rows_after = len(final_dataset)

print(f"Removed {rows_before - rows_after} rows with missing dates.")
print(f"Final dataset shape is now: {final_dataset.shape}")
# 8. Export
output_path = "../data/final/final_dataset.csv"
final_dataset.to_csv(output_path, index=False)
print(f"\nDataset exported successfully to: {output_path}")

# 9. Sample preview
print("\nSample of exported data:")
display(final_dataset.sample(5, random_state=42))

Final dataset ready for export.
Shape: (72053, 27)
Columns included: 27
Rolling features detected: 12

Columns with missing values:
home_gameDate    174
dtype: int64
Removed 174 rows with missing dates.
Final dataset shape is now: (71879, 27)

Dataset exported successfully to: ../data/final/final_dataset.csv

Sample of exported data:


,home_win,score_diff,diff_points,diff_assists,diff_reboundsTotal,diff_steals,diff_blocks,diff_fieldGoalsPercentage,diff_threePointersPercentage,diff_freeThrowsPercentage,is_playoffs,is_regular,home_teamName,away_teamName,home_gameDate,home_points_rolling5,home_assists_rolling5,home_reboundsTotal_rolling5,home_fieldGoalsPercentage_rolling5,home_threePointersPercentage_rolling5,home_freeThrowsPercentage_rolling5,away_points_rolling5,away_assists_rolling5,away_reboundsTotal_rolling5,away_fieldGoalsPercentage_rolling5,away_threePointersPercentage_rolling5,away_freeThrowsPercentage_rolling5
66878,1,12.0,12.0,2.0,1.0,0.0,3.0,0.087,0.032,-0.023,0,0,Jazz,Timberwolves,2021-12-23 21:00:00,111.4,29.6,44.8,0.4732,0.3264,0.7476,110.8,22.0,40.6,0.4442,0.3600,0.8060
36704,1,9.0,9.0,2.0,24.0,0.0,0.0,0.024,0.055,-0.135,0,0,Magic,Pistons,1999-11-03 19:30:00,99.8,19.8,46.6,0.4436,0.3546,0.7980,87.2,17.4,38.6,0.4190,0.2718,0.7684
44855,1,13.0,13.0,6.0,4.0,11.0,0.0,-0.006,-0.378,0.036,0,0,Hornets,Spurs,2005-12-18 19:00:00,89.2,14.6,41.0,0.4070,0.2416,0.8360,97.8,21.6,43.8,0.4878,0.4278,0.6688
71335,0,-21.0,-21.0,-10.0,-13.0,0.0,-3.0,-0.057,0.040,-0.078,0,0,Bulls,Warriors,2025-02-08 20:00:00,118.6,25.8,43.6,0.4894,0.3924,0.7970,114.8,28.4,43.2,0.4438,0.3924,0.7270
39414,1,10.0,10.0,-2.0,-11.0,0.0,-4.0,0.057,-0.056,0.012,0,0,Mavericks,Warriors,2001-11-27 20:30:00,91.2,19.2,34.4,0.4358,0.2404,0.7576,93.2,18.8,48.4,0.4320,0.4194,0.7316
